In [11]:
import numpy as np
import scipy.linalg
import time

In [12]:
n = 2048
#A = np.tril(-1*np.ones(n),-1)
#A[:,n-1] = 1
#np.fill_diagonal(A,1)
A = -100 + 200*np.random.rand(n,n)
x_known = np.ones((n,1))
#x_known = -100 + 200*np.random.rand(n,1)
b = A @ x_known
#A = np.asfortranarray(A)
LU = np.matrix(np.copy(A))
L = np.eye(n)
P = np.arange(n)
Q = np.arange(n)

t = time.time()
for i in range(0,n-1):
    x_ = np.abs(LU[i:,i:].A1)
    p = np.argmax(x_)
    p_row,p_col = np.unravel_index(p,LU[i:,i:].shape)
    P[[i,p_row+i]] = P[[p_row+i,i]] #interchange elements
    Q[[i,p_col+i]] = Q[[p_col+i,i]] #interchange elements
    LU[[i,p_row+i],:] = LU[[p_row+i,i],:] #interchange rows
    LU[:,[i,p_col+i]] = LU[:,[p_col+i,i]] #interchange columns
    
    l = LU[i+1:n,i]/LU[i,i]
    LU[i+1:n,i:n] = LU[i+1:n,i:n] - l*LU[i,i:n]
    LU[i+1:n,i] = l
    
time.time() - t  

16.781253576278687

In [13]:
U = np.triu(LU)
L = np.tril(LU,k=-1)
np.fill_diagonal(L, 1)

b_LU = b[P]
x = np.zeros((n,1));
y = np.zeros((n,1));
z = np.zeros((n,1));

for i in range(0,n):
    z[i] = b_LU[i] - np.inner(L[i,:],z.T)
 
y[n-1] = z[n-1]/U[n-1,n-1] 
for i in range(n-2,-1,-1):
    y[i] = (z[i] - np.inner(U[i,i+1:],y[i+1:].T))/U[i,i]

I = np.eye(n,n)
I = I[:,Q]
x = I@y

normi1 = np.linalg.norm(x-x_known,np.inf)
normi2 = np.linalg.norm(A@x-b,np.inf)
normi1, normi2

(2.0874413309002193e-12, 1.7280399333685637e-10)